In [212]:
# import packages

import pandas_datareader.data as web
import datetime
import pandas as pd
import numpy as np
import scipy.stats as stat

In [213]:
# set up the parameters

start = datetime.datetime(2010, 1, 1)
end = datetime.datetime(2016, 1, 1)

v0 = 1000000   # initial investment

tickers_string = "AAPL,AMZN,MSFT,GOOG"   # a string; as an example, tickers separated by comma.
tickers_list = tickers_string.split(",")  # split the string by the commas into a list of strings
tickers_num = len(tickers_list)

weight_string = "0.25,0.25,0.25,0.25"
weight_list = map(float, weight_string.split(","))  
# split the string by the commas, and map each string in the resulting list into a float

VaR_prob = 0.99
ES_prob = 0.975
window = 2 # Using x years historical data for estimation
window_days = window * 252   
horizon_days = 5 # x days VaR/ES
horizon = horizon_days/252

In [214]:
d={}
for ticker in tickers_list:
    d["{0}".format(ticker)] = web.DataReader(ticker, 'yahoo', start, end)['Adj Close'].rename(ticker)
df = pd.DataFrame(d).sort_index(ascending = False)       
# create a pandas dataframe where each column is the Adjusted Close price for the respective ticker

In [215]:
# Calculate estimated parameters based on x year rolling windows
def gbm_est(prices, window_days):
    rtn = -np.diff(np.log(prices))
    rtnsq = rtn * rtn
    mubar = list(reversed(np.convolve(rtn, np.ones((window_days,))/window_days, mode='valid')))
    x2bar = list(reversed(np.convolve(rtnsq, np.ones((window_days,))/window_days, mode='valid')))
    var = x2bar - np.square(mubar)
    sigmabar = np.sqrt(np.maximum(var, np.zeros(len(var))))
    sigma = sigmabar / np.sqrt(1/252)
    mu = np.array(mubar)*252 + np.square(sigma)/2
    return rtn, mu, sigma, mubar, sigmabar

In [216]:
# test with AAPL 
x = df["AAPL"]
rtn, mu, sigma, mubar, sigmabar = gbm_est(x, window_days)

In [217]:
# Calculate VaR assuming gbm 
def gbm_VaR(v0, mu, sigma, p, t):
    VaR = v0 - v0 * np.exp(sigma * np.sqrt(t) * stat.norm.ppf(1-p) + (mu - np.square(sigma)/2) * t)
    return VaR

In [218]:
VaR = gbm_VaR(v0, mu, sigma, VaR_prob, horizon)

In [234]:
VaR

array([ 76980.60738557,  76952.01066241,  76644.70878177, ...,
        73332.18067359,  73385.25958295,  73757.91043265])

In [231]:
# Calculate ES assuming gbm
def gbm_ES(v0, mu, sigma, p, t):
    ES = v0 * (1 - np.array(stat.norm.cdf(stat.norm.ppf(1-p) - np.sqrt(t)*sigma)) * np.array(np.exp(mu*t)/(1-p)))
    return ES

In [235]:
ES = gbm_ES(v0, mu, sigma, ES_prob, horizon)

In [236]:
ES

array([ 77299.95452569,  77271.38203784,  76964.00403671, ...,
        73632.3490151 ,  73685.48844844,  74058.30004314])